In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import pandas as pd
import nltk as nltk

from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams
from gensim import corpora

In [ ]:
# Load the BoW features as Pandas dataframe
bows_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/courses_bows.csv"
bows_df = pd.read_csv(bows_url)
bows_df = bows_df[['doc_id', 'token', 'bow']]

In [ ]:
bows_df.head(10)

,doc_id,token,bow
0,ML0201EN,ai,2
1,ML0201EN,apps,2
2,ML0201EN,build,2
3,ML0201EN,cloud,1
4,ML0201EN,coming,1
5,ML0201EN,create,1
6,ML0201EN,data,1
7,ML0201EN,developer,1
8,ML0201EN,found,1
9,ML0201EN,fun,1


In [ ]:
# Load the course dataframe
course_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/course_processed.csv"
course_df = pd.read_csv(course_url)

In [ ]:
course_df.head(10)

,COURSE_ID,TITLE,DESCRIPTION
0,ML0201EN,robots are coming build iot apps with watson ...,have fun with iot and learn along the way if ...
1,ML0122EN,accelerating deep learning with gpu,training complex deep learning models with lar...
2,GPXX0ZG0EN,consuming restful services using the reactive ...,learn how to use a reactive jax rs client to a...
3,RP0105EN,analyzing big data in r using apache spark,apache spark is a popular cluster computing fr...
4,GPXX0Z2PEN,containerizing packaging and running a sprin...,learn how to containerize package and run a ...
5,CNSC02EN,cloud native security conference data security,introduction to data security on cloud
6,DX0106EN,data science bootcamp with r for university pr...,a multi day intensive in person data science ...
7,GPXX0FTCEN,learn how to use docker containers for iterati...,learn how to use docker containers for iterati...
8,RAVSCTEST1,scorm test 1,scron test course
9,GPXX06RFEN,create your first mongodb database,in this guided project you will get started w...


In [ ]:
ml_course = bows_df[bows_df['doc_id'] == 'ML0101ENv3']
ml_course

,doc_id,token,bow
2747,ML0101ENv3,course,1
2748,ML0101ENv3,learning,4
2749,ML0101ENv3,machine,3
2750,ML0101ENv3,need,1
2751,ML0101ENv3,get,1
2752,ML0101ENv3,started,1
2753,ML0101ENv3,python,2
2754,ML0101ENv3,tool,1
2755,ML0101ENv3,tools,1
2756,ML0101ENv3,predict,1


In [ ]:
ml_courseT = ml_course.pivot(index=['doc_id'], columns='token').reset_index(level=[0])
ml_courseT

doc_id        bow                                              \
token             beneficial course free future get give hidden insights   
0      ML0101ENv3          1      1    1      1   1    1      1        1   

                                                                          \
token learning machine need predict python started supervised tool tools   
0            4       3    1       1      2       1          1    1     1   

                           
token trends unsupervised  
0          1            1

In [ ]:
def pivot_two_bows(basedoc, comparedoc):
    base = basedoc.copy()
    base['type'] = 'base'
    compare = comparedoc.copy()
    compare['type'] = 'compare'
    # Append the two token sets vertically
    join = base.append(compare)
    # Pivot the two joined courses
    joinT = join.pivot(index=['doc_id', 'type'], columns='token').fillna(0).reset_index(level=[0, 1])
    # Assign columns
    joinT.columns = ['doc_id', 'type'] + [t[1] for t in joinT.columns][2:]
    return joinT

In [ ]:
course1 = bows_df[bows_df['doc_id'] == 'ML0151EN']
course2 = bows_df[bows_df['doc_id'] == 'ML0101ENv3']

In [ ]:
bow_vectors = pivot_two_bows(course1, course2)
bow_vectors

<ipython-input-58-abf8720ec68f>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)


,doc_id,type,approachable,basics,beneficial,comparison,course,dives,free,future,...,relates,started,statistical,supervised,tool,tools,trends,unsupervised,using,vs
0,ML0101ENv3,compare,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,ML0151EN,base,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0


In [ ]:
similarity = 1 - cosine(bow_vectors.iloc[0,2:], bow_vectors.iloc[1,2:])
similarity

0.6626221399549089

In [ ]:
def similarity_two_bows(basedoc, comparedoc):
    base = basedoc.copy()
    base['type'] = 'base'
    compare = comparedoc.copy()
    compare['type'] = 'compare'
    # Append the two token sets vertically
    join = base.append(compare)
    # Pivot the two joined courses
    joinT = join.pivot(index=['doc_id', 'type'], columns='token').fillna(0).reset_index(level=[0, 1])
    # Assign columns
    joinT.columns = ['doc_id', 'type'] + [t[1] for t in joinT.columns][2:]
    similarity = 1 - cosine(joinT.iloc[0,2:], joinT.iloc[1,2:])
    return similarity

courses = course_df.COURSE_ID
ml_course = bows_df[bows_df['doc_id'] == 'ML0101ENv3']

similarity_score_dict = {
    'Course_ID' : [],
    'Course_Description' : [],
    'Similarity_ML0101ENv3' : []
}

for course in courses:
    bow_course = bows_df[bows_df['doc_id'] == course]
    similarity_score = similarity_two_bows(ml_course, bow_course)
    desc = course_df[course_df['COURSE_ID'] == course].DESCRIPTION
    similarity_score_dict['Course_ID'].append(course)
    similarity_score_dict['Course_Description'].append(desc)
    similarity_score_dict['Similarity_ML0101ENv3'].append(similarity_score)

similarity_df = pd.DataFrame.from_dict(similarity_score_dict)


<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-i

      Course_ID                                 Course_Description  \
0      ML0201EN  0    have fun with iot and learn along the way...   
1      ML0122EN  1    training complex deep learning models wit...   
2    GPXX0ZG0EN  2    learn how to use a reactive jax rs client...   
3      RP0105EN  3    apache spark is a popular cluster computi...   
4    GPXX0Z2PEN  4    learn how to containerize  package  and r...   
..          ...                                                ...   
302  excourse89  302    in this course  we ll look at the javas...   
303  excourse90  303    learn foundational programming concepts...   
304  excourse91  304    this course explores javascript based f...   
305  excourse92  305    this course is designed to start you on...   
306  excourse93  306    this course is the third in our javascr...   

     Similarity_ML0101ENv3  
0                 0.000000  
1                 0.261265  
2                 0.000000  
3                 0.000000  
4             

<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-input-62-980e8ae52fe8>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  join = base.append(compare)
<ipython-i

In [ ]:
similar_course = similarity_df[similarity_df['Similarity_ML0101ENv3']>0.5]
similar_course

,Course_ID,Course_Description,Similarity_ML0101ENv3
157,ML0109EN,157 machine learning dimensionality reduc...,0.521749
158,ML0101ENv3,158 machine learning can be an incredibly b...,1.000000
200,ML0151EN,200 this machine learning with r course div...,0.662622
259,excourse46,259 machine learning is the science of gett...,0.612054
260,excourse47,260 machine learning often called artifici...,0.634755
273,excourse60,273 if you are a software developer who wan...,0.549040
